# Download the Canvas API reference from Instructure
#### This isn't my code, I found it at https://gitlab.unimelb.edu.au/dsweeney/canvas-api-postman/-/blob/master/canvas-api-postman.py
#### I just changed the output filename to contain the day when the API was downloaded.
#### When you run this, it will download all the API documentation and examples to one big JSON file, which you can then download and import into PostMan
#### We're doing this in JupyterHub so you do not have to install Python just for this one task

### To download the API into a format the PostMan can ingest, click "Cell -> Run All" in the menu above.

In [4]:
import json
import requests
import re
from datetime import datetime


base_url = 'https://canvas.instructure.com/'
postman_domain = '{{domain}}'
file_name = datetime.today().strftime('%Y-%m-%d')+'_CanvasAPIinPostManFormat.json'

print('Getting base doc')
top_level_docs = requests.get('{}/doc/api/api-docs.json'.format(base_url))

# get the API portion of the JSON object returned from Canvas
docs = top_level_docs.json()['apis']

# create a framework JSON object to collect the different APIs
postman_collection = {
	'info': {
		'name': 'Canvas API',
		'schema': 'https://schema.getpostman.com/json/collection/v2.1.0/collection.json'
	},
	'item': [],
	'auth': {
		'type': 'bearer',
		'bearer': [
			{
				'key': 'token',
				'value': '{{user_token}}',
				'type': 'string'
			}
		]
	}
}

# loop through each of the APIs
# see here for the full list: https://canvas.instructure.com/doc/api/api-docs.json
for doc in docs:
	folder = {
		'name': doc['description'],
		'item': []
	}

	# get the JSON object for this API endpoint
	print('Getting {} doc'.format(folder['name']))
	raw_doc_request = requests.get('{}/doc/api{}'.format(base_url,doc['path']))
	doc_detail = raw_doc_request.json()

	# loop though the different calls for this API
	for item in doc_detail['apis']:
		# construct the path (including path variables)
		path_array = [
			'api'
			]
		path_array_items = []
		path_elements = re.findall('(\/[^/]*)', item['path'])
		for el in path_elements:
			el = el[1:]
			# path variables are translated here from {course_id} to :course_id
			if '{' in el:
				path_array_items = re.sub('{',':',re.sub('}','',el))
				path_array.append(path_array_items)
			else:
				path_array.append(el)
		revised_path = '/'.join(path_array)

		# form parameters appear in the 'body' section in Postman
		form_parameters = []
		# variables appear as path variables in Postman in the Params section
		variable_array = []
		if 'parameters' in list(item['operations'][0]):
			for param in item['operations'][0]['parameters']:
				if param['paramType'] == 'form':
					param_type = 'text'
					if param['type'] == 'file':
						param_type = 'file'

					temp_param = {
						'key': param['name'],
						'value': '',
						'type': param_type,
						'description': param['description']
					}
					form_parameters.append(temp_param)
				elif param['paramType'] == 'path':
					temp_var = {
						'key': param['name'],
						'description': param['description']
					}
					variable_array.append(temp_var)

		# put it all together
		api_call = {
			'name': item['operations'][0]['summary'],
			'request': {
				'method': item['operations'][0]['method'],
				'header': [],
				'url': {
					'raw': 'https://{}/api/{}'.format(postman_domain, revised_path),
					'protocol': 'https',
					'host': [ postman_domain ],
					'path': path_array,
					'variable': variable_array
				},
				'description': item['description']
			},
			'response': []
		}

		if len(form_parameters) > 0:
			api_call['request']['body'] = {
				'mode': 'formdata',
				'formdata': form_parameters
			}
		folder['item'].append(api_call)
	postman_collection['item'].append(folder)

# write the collection to a file
with open(file_name, 'w') as f:
	json.dump(postman_collection, f, indent=3, separators=(',', ': '))


Getting base doc
Getting Account Calendars doc
Getting Account Domain Lookups doc
Getting Account Notifications doc
Getting Account Reports doc
Getting Accounts doc
Getting Accounts (LTI) doc
Getting Admins doc
Getting Analytics doc
Getting Announcement External Feeds doc
Getting Announcements doc
Getting API Token Scopes doc
Getting Appointment Groups doc
Getting Assignment Extensions doc
Getting Assignment Groups doc
Getting Assignments doc
Getting Authentication Providers doc
Getting Authentications Log doc
Getting Blackout Dates doc
Getting Blueprint Courses doc
Getting Bookmarks doc
Getting Brand Configs doc
Getting Calendar Events doc
Getting Collaborations doc
Getting CommMessages doc
Getting Communication Channels doc
Getting Conferences doc
Getting Content Exports doc
Getting Content Migrations doc
Getting Content Security Policy Settings doc
Getting Content Shares doc
Getting Conversations doc
Getting Course Audit log doc
Getting Course Quiz Extensions doc
Getting Courses doc